For this problem, we don't care if a search is booked or not so we can use the data from the booking_data file to have a bigger dataset.

In [ ]:
import pandas as pd
import numpy as np
import datetime as date
from pytz import timezone
import re
import warnings
warnings.filterwarnings('ignore')


df_search = pd.read_csv('search_data.csv')
df_booking = pd.read_csv('booking_data.csv')
df_search = df_search.drop_duplicates()
df_booking = df_booking.drop_duplicates()
df_booking=df_booking.rename(columns = {'booking_id':'search_id','booking_user_id':'search_user_id','booking_time':'search_time','arrial_datetime_2':'arrival_datetime_2'})
df_search = pd.concat([df_search,df_booking])

First we clear out some inconsistencies in the data such as a search with 0 passengers but an airfare. 
We also regroup the category of cabin_class into more coherent classes and we transform all the airfares into the same currency.
I only managed to find a simple external dataset of principal (ie only 33) currencies exchange rate to euro so I left out the rows with a currency that was not in this external dataset.

In [ ]:
df_search = df_search[df_search.passengers != 0]

df_search['cabin_class'] = df_search['cabin_class'].map({'economy':'economy', 'Economy':'economy', 'Business':'business', 'mixed':'mixed', 'First':'first', 'business':'business',
 'premium_economy':'premium_economy', 'first':'first', 'Premium Economy':'premium_economy', 'First Class':'first', 'COACH':'economy',
 'premiumEconomy':'premium_economy', 'ECONOMY':'economy'})

df_exrate = pd.read_csv('eurofxref.csv')

df_exrate.columns = [name.strip() for name in df_exrate.columns]
df_exrate.drop('',1)
df_principal_currency = df_search[df_search.currency.isin(list(df_exrate.columns))]
sLength = len(df_principal_currency['currency'])
df_principal_currency['exchange']=df_principal_currency.apply(lambda df: df_exrate[df.currency][0], axis=1)
df_principal_currency['fare_eur']=df_principal_currency.fare/df_principal_currency.exchange


Now we can start to engineer our features.
I first focused on the time of search and the time flight itinerary.
I got 11 features out of it:
- day of the week of the search
- day of the year of the search
- day of the week of the departure for the trip
- day of the year of the departure for the trip
- day of the week of the return from the trip (if applicable)
- day of the year of the return from the trip (if applicable)
- difference in days between date of search and date of departure
- difference in days between date of search and date of return   
- if the trip is returning or not
- number of flights for the departure itinerary
- number of flights for the return itinerary
Since there is no timezone for the search datetime (only a country looking at the pos column), I didn't calculated the features with UTC offsets at all (even if I could have computed the UTC offset for the departure or return datetimes with the airport data).

In [ ]:
def diffDays(df):
    return (df['departure_date'].date()-df['search_date'].date()).days

df = df_principal_currency
df['search_date']=df['search_time'].apply(lambda x:date.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
df['departure_time'] = df['departure_datetime_1'].apply(lambda x:x.split(";")[0])
df['departure_date']=df['departure_time'].apply(lambda x:date.datetime.strptime(x[:19],"%Y-%m-%dT%H:%M:%S"))
df['days_to_departure']=df.apply(diffDays,axis=1)
df['search_day_of_year']=df['search_date'].apply(lambda x:x.timetuple().tm_yday)
df['search_day_of_week']=df['search_date'].apply(lambda x:x.timetuple().tm_wday)
df['departure_day_of_week']=df['departure_date'].apply(lambda x:x.timetuple().tm_wday)
df['departure_day_of_year']=df['departure_date'].apply(lambda x:x.timetuple().tm_yday)
df['departure_day_of_week']=df['departure_date'].apply(lambda x:x.timetuple().tm_wday)

def splitPossibleNull(x):
    if x and str(x)!='nan': 
        return x.split(";")[0]
    else:
        return "2001-01-01T00:00:00"
def diffDaysReturn(df):
    if int(df['return_date'].year) >= int(df['search_date'].year):
        return (df['return_date'].date()-df['search_date'].date()).days
    else:
        #print(df['return_date'].year)
        return 0

df['return_time'] = df['arrival_datetime_1'].apply(splitPossibleNull)
df['return_date']=df['return_time'].apply(lambda x:date.datetime.strptime(x[:19],"%Y-%m-%dT%H:%M:%S"))
df['days_to_return']=df.apply(diffDaysReturn,axis=1)
df['return_day_of_year']=df['return_date'].apply(lambda x:x.timetuple().tm_yday)
df['return_day_of_week']=df['return_date'].apply(lambda x:x.timetuple().tm_wday)
df['has_return']=df['arrival_datetime_1'].apply(lambda x: float(str(x)!='nan'))

def numFlightsReturn(x):
    if x and str(x)!='nan': 
        return len(x.split(";"))
    else:
        return 0
    
df['nb_flights_departure']=df['flight_num_1'].apply(lambda x: len(x.split(';')))
df['nb_flights_return']=df['flight_num_2'].apply(numFlightsReturn)




Next I tried to compute the distance between the first airport of the itinerary and the last one. But when I looked at the airport data, there were a lot of airports for which there was no latitude or longitude or altitude. I tried to get an external dataset but in the one I found not all the airports were mentioned so I gave up on this idea and only processed a distance code represented by the concatenation of the iata code of the first airport of the itinerary and the iata code of the last one.

In [ ]:
def joinCoord(df_airport):
    return ";".join((str(df_airport['lat']),str(df_airport['lon']),str(df_airport['alt'])))

def codeDistance(df):
    airports = sorted([df['origin'],df['destination']], key=str.lower)
    return ''.join(airports)

df_airport = pd.read_csv('airports.csv',header=None, index_col=None)
df_airport = df_airport[[4,6,7,8]]
df_airport.columns = ['iata_code','lat','lon','alt']
df_airport['coord']= df_airport.apply(joinCoord,axis=1)
df_airport.set_index('iata_code', drop=True, inplace=True)
df_airport = df_airport[['coord']]
dictionary = df_airport.to_dict(orient="index")
#df['coord_origin']=df['origin'].apply(lambda x:dictionary[x])
#df['coord_destination']=df['destination'].apply(lambda x:dictionary[x])

df['distance_code'] = df.apply(codeDistance,axis=1)
print(df['distance_code'][:10])

Now we can gather all the features that will be used for the regression model ie the features previously engineered and also:
- the partner_id
- the number of passengers
- the number of requests
- the supplier
- the cabin class
- the airfare in euro (will serve as target values for the model)
The resulting dataframe is saved in a csv file.

In [ ]:
df_feature = df[['search_id','partner_id','passengers','num_requests','fare_eur','supplier','cabin_class',
                 'days_to_departure','search_day_of_year','search_day_of_week','departure_day_of_year','departure_day_of_week',
                 'days_to_return','return_day_of_year','return_day_of_week','has_return','nb_flights_departure','nb_flights_return',
                 'distance_code']]

df_feature.to_csv('features_regression.csv')